#### функция calculate_downtime для каждого инцидента:
1. получает на вход список отсечек от прошлой функции
2. проходит последовательно по всем отсечкам и накапливает время простоя

In [37]:
#импорты
import pandas as pd
import re
import datetime as dt
from datetime import timedelta
import time
#from datetime import datetime, timedelta, date, time

Используем тестовый датасет с синтетическими данными. Колонка working_days содержит информацию о рабочих днях недели, где 1 - понедельник, а 7 - воскресенье. Колонка working_hours содержит часы работы по каждому дню недели (разделение символом |, обеденные перерывы указаны в круглых скобках). Время начала простоя отражено в колонке downtime_start, а  время окончания - в downtime_finish.

In [38]:
# загружаем данные по инцидентам
df = pd.read_excel('Тестовый датасет.xlsx', 'Sheet1')

In [39]:
df

,N,working_days,working_time,downtime_start,downtime_finish
0,0,123456,08:30 18:00|08:30 18:00|08:30 18:00|08:30 18:0...,2023-11-24 08:04:20,2023-12-07 10:42:51
1,1,12345,09:00 18:00|09:00 19:00|09:00 19:00|09:00 18:0...,2023-11-24 06:34:55,2023-12-11 10:24:23
2,2,123456,10:00 19:00|10:00 19:00|10:00 19:00|10:00 19:0...,2023-12-13 09:55:25,2023-12-27 09:53:08
3,3,123456,08:30 19:00|08:30 19:00|08:30 19:00|08:30 19:0...,2023-12-08 09:21:28,2023-12-12 06:22:39
4,4,23456,08:30 18:00|08:30 18:00|08:30 18:00|08:30 18:0...,2023-12-05 07:26:00,2023-12-20 12:14:26
...,...,...,...,...,...
9996,9996,123456,09:00 20:00|09:00 20:00|09:00 20:00|09:00 20:0...,2023-12-04 05:33:26,2023-12-04 13:50:00
9997,9997,123456,09:00 19:00|09:00 19:00|09:00 19:00|09:00 19:0...,2023-12-04 11:58:42,2023-12-04 15:55:00
9998,9998,123456,09:00 18:30|09:00 18:30|09:00 18:30|09:00 18:3...,2023-12-05 07:21:12,2023-12-06 04:47:00
9999,9999,123456,08:30 19:30|08:30 19:30|08:30 19:30|08:30 19:3...,2023-12-16 00:17:30,2023-12-16 01:55:55


####  подготовим данные
Как всегда, самое сложное  это подготовка данных. В статье представлены разнообразные варианты графиков.  Эта часть совсем не универсальна и очень зависит от того, в какой форме мы получили сходные данные. В датасет включен первый вариант из статьи, подготовим эти данные к работе.

Поскольку время в графике не привязано к конкретному дню календаря, его можно зафиксировать в формате time, но для наших целей сразу представим как timedelta.

In [62]:
def add_schedule(row):
    
    schedule = [[], [], [], [], [], [], []] # создадим пустой график
    
    working_day = len(str(row['working_days']))
    initial_schedule = str(row['working_time'])
    for day in range(working_day):
        weekday_number = int((str(row['working_days']))[day])-1
        match = (initial_schedule.split('|')[day]).split()
        if match != []: # для каждого непустого дня 
            # время начала
            try:
                from_time = timedelta(hours = int(match[0].split(':')[0]), minutes = int(match[0].split(':')[1]))
                from_time_dict = {'time': from_time,
                                  'flag': True,
                                  'start_finish': ''}
            except:
                from_time_dict = ''
            
            # время окончания
            try:
                to_time = timedelta(hours = int(match[1].split('(')[0].split(':')[0]), minutes = int(match[1].split('(')[0].split(':')[1]))
                to_time_dict = {'time': to_time,
                                'flag': False,
                                'start_finish': ''}
            except:
                to_time_dict = ''
                
                   
            if '(' in match[1]:   
                break_variable = (str(initial_schedule.split('|')[day]).split('(')[1]).replace(')', '')
                break_start = timedelta(hours = int(break_variable.split()[0].split(':')[0]), minutes = int(break_variable.split()[0].split(':')[1]))
                break_start_dict = {'time': break_start,
                                'flag': False,
                                'start_finish': ''}
               
                break_finish = timedelta(hours = int(break_variable.split()[1].split(':')[0]), minutes = int(break_variable.split()[1].split(':')[1]))
                break_finish_dict = {'time': break_finish,
                                  'flag': True,
                                  'start_finish': ''}
            else:
                break_start_dict = ''
                break_finish_dict = ''
            schedule[weekday_number] = [from_time_dict, to_time_dict, break_start_dict, break_finish_dict]
 
    return schedule

In [63]:
df['schedule'] = df.apply(add_schedule, axis = 1)

In [64]:
df

,N,working_days,working_time,downtime_start,downtime_finish,schedule
0,0,123456,08:30 18:00|08:30 18:00|08:30 18:00|08:30 18:0...,2023-11-24 08:04:20,2023-12-07 10:42:51,"[[{'time': 8:30:00, 'flag': True, 'start_finis..."
1,1,12345,09:00 18:00|09:00 19:00|09:00 19:00|09:00 18:0...,2023-11-24 06:34:55,2023-12-11 10:24:23,"[[{'time': 9:00:00, 'flag': True, 'start_finis..."
2,2,123456,10:00 19:00|10:00 19:00|10:00 19:00|10:00 19:0...,2023-12-13 09:55:25,2023-12-27 09:53:08,"[[{'time': 10:00:00, 'flag': True, 'start_fini..."
3,3,123456,08:30 19:00|08:30 19:00|08:30 19:00|08:30 19:0...,2023-12-08 09:21:28,2023-12-12 06:22:39,"[[{'time': 8:30:00, 'flag': True, 'start_finis..."
4,4,23456,08:30 18:00|08:30 18:00|08:30 18:00|08:30 18:0...,2023-12-05 07:26:00,2023-12-20 12:14:26,"[[], [{'time': 8:30:00, 'flag': True, 'start_f..."
...,...,...,...,...,...,...
9996,9996,123456,09:00 20:00|09:00 20:00|09:00 20:00|09:00 20:0...,2023-12-04 05:33:26,2023-12-04 13:50:00,"[[{'time': 9:00:00, 'flag': True, 'start_finis..."
9997,9997,123456,09:00 19:00|09:00 19:00|09:00 19:00|09:00 19:0...,2023-12-04 11:58:42,2023-12-04 15:55:00,"[[{'time': 9:00:00, 'flag': True, 'start_finis..."
9998,9998,123456,09:00 18:30|09:00 18:30|09:00 18:30|09:00 18:3...,2023-12-05 07:21:12,2023-12-06 04:47:00,"[[{'time': 9:00:00, 'flag': True, 'start_finis..."
9999,9999,123456,08:30 19:30|08:30 19:30|08:30 19:30|08:30 19:3...,2023-12-16 00:17:30,2023-12-16 01:55:55,"[[{'time': 8:30:00, 'flag': True, 'start_finis..."


#### последовательно запустим 2 функции: сформируем осортированный список временных отсечек и посчитаем по нему простои

In [69]:
# функция сбора и сортировки флагов
def collect_downtime (row):
    start = pd.to_datetime(row['downtime_start'], format = '%d.%m.%Y %H:%M:%S')
    finish = pd.to_datetime(row['downtime_finish'], format = '%d.%m.%Y %H:%M:%S')
    schedule = row['schedule']
    
# список всех дней между точками А и В
    day_list = pd.date_range(start.date(), finish.date()).tolist()
# список, куда мы сложим все флаги
    target_list_full = []
    
# обрабатываем каждый день из списка
    for day in day_list:
        target_list_day = [] # переменная для накопления флагов внутри дня
        day_begin = day.replace(hour=0, minute = 0, second=0, microsecond=0)
        day_of_week = day.dayofweek # день недели
        day_schedule = schedule[day_of_week] # график интересующего нас дня
        
#обрабатываем каждую временную отсечку внутри дня. Здесь мы складываем время начала дня с timedelta из графика
        for element in day_schedule:
            try:
                target_element = element
                target_element['time'] += day_begin
                target_list_day.append(target_element)
            except:
                pass
            
        target_list_full += target_list_day

# добавим в список начало и конец инцидента, началу сразу присвоим флаг False
    target_begin = {'time': start,
                    'flag': False,
                    'start_finish': 'Start'}
    
    target_end = {'time': finish,
                  'flag': '',
                  'start_finish': 'Finish'}
    
    target_list_full.append(target_begin)
    target_list_full.append(target_end)

# ключевой момент: сортируем список флагов по дате
    target_list_full = sorted(target_list_full, key = lambda x: x['time'])
    
    result_dict = {'target_list' : target_list_full,
                   'begin_index' : target_list_full.index(target_begin),
                   'end_index' : target_list_full.index(target_end)}
    return result_dict

In [70]:
df['result_dict'] = df.apply(collect_downtime, axis = 1)


In [75]:
# функция расчета рабочего времени. На вход - словарь, результат функции collect_downtime
def calculate_downtime (row):
    target_list = row['result_dict'].get('target_list')
    begin_index = row['result_dict'].get('begin_index')
    end_index = row['result_dict'].get('end_index')
    time_sum = dt.timedelta(0) # переменная для накопления времени
     
    #надо проставить метки True/False для начала и конца инцидента
    # если точка А не первая в списке, тогда ориентируемся на предудущюю отсечку:
    if begin_index != 0:
        target_list[begin_index]['flag'] = target_list[0]['flag']
    # точка А первая в списке, точка B не вторая:    
    elif (end_index != 1):
        target_list[begin_index]['flag'] = not (target_list[1]['flag'])
    # точка А первая в списке, точка B вторая:
    else:
        try:
            target_list[begin_index]['flag'] = not (target_list[2]['flag'])
        except:
            pass

    # теперь у всех отсечек, кроме финишной, есть признак True/False. Пройдем последовательно по всем отсечкам
    for i in range(end_index - begin_index):
        time_period = dt.timedelta(0) # переменная для накопления времени простоя
        if target_list[i+ begin_index]['flag'] == True:
            time_period = target_list[i+1+begin_index]['time'] - target_list[i+begin_index]['time']
        time_sum += time_period 
    
    return round(time_sum.total_seconds() / 60, 0) #сразу преобразуем в минуты

In [76]:
df['calculate_downtime'] = df.apply(calculate_downtime, axis = 1)

In [77]:
df.to_excel('semiresult.xlsx', sheet_name='1')